In [2]:
import json, math, random, os
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple

DATA_DIR = "../data"
SEED_PATH = f"{DATA_DIR}/books_seed.json"
STATE_PATH = f"{DATA_DIR}/books_state.json"

K_FACTOR_DEFAULT = 32  # can tune later by activity
ELO_START = 1200


In [3]:
@dataclass
class Book:
    id: str
    title: str
    author: str
    rating: float = ELO_START
    wins: int = 0
    losses: int = 0
    matches: int = 0

def load_seed(path: str) -> Dict[str, Book]:
    with open(path, "r", encoding="utf-8") as f:
        items = json.load(f)
    books = {}
    for x in items:
        books[x["id"]] = Book(id=x["id"], title=x["title"], author=x["author"])
    return books

def load_state(path: str) -> Dict[str, Book]:
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)
    return {k: Book(**v) for k, v in raw.items()}

def save_state(books: Dict[str, Book], path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    payload = {k: asdict(v) for k, v in books.items()}
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)


In [4]:
if os.path.exists(STATE_PATH):
    books = load_state(STATE_PATH)
else:
    books = load_seed(SEED_PATH)
    save_state(books, STATE_PATH)

len(books), list(books.keys())[:3]


(8, ['b1', 'b2', 'b3'])

In [5]:
def expected_score(ra: float, rb: float) -> float:
    # Probability A beats B
    return 1.0 / (1.0 + 10 ** ((rb - ra) / 400.0))

def update_elo(ra: float, rb: float, outcome_a: float, k: float = K_FACTOR_DEFAULT) -> Tuple[float, float]:
    """
    outcome_a: 1 if A wins, 0 if A loses (0.5 for draw if ever needed)
    Returns new (ra, rb)
    """
    ea = expected_score(ra, rb)
    eb = 1.0 - ea
    ra_new = ra + k * (outcome_a - ea)
    rb_new = rb + k * ((1.0 - outcome_a) - eb)
    return ra_new, rb_new


In [7]:
assert abs(expected_score(1200, 1200) - 0.5) < 1e-9
ra, rb = update_elo(1200, 1200, outcome_a=1)
assert ra > 1200 and rb < 1200


In [8]:
def select_pair(books: Dict[str, Book]) -> Tuple[Book, Book]:
    a, b = random.sample(list(books.values()), 2)
    return a, b

# Optional: near-rating selection (window in Elo points)
def select_pair_nearby(books: Dict[str, Book], window: int = 200) -> Tuple[Book, Book]:
    pool = list(books.values())
    a = random.choice(pool)
    candidates = [x for x in pool if x.id != a.id and abs(x.rating - a.rating) <= window]
    if not candidates:
        candidates = [x for x in pool if x.id != a.id]
    b = random.choice(candidates)
    return (a, b) if random.random() < 0.5 else (b, a)


In [9]:
def apply_vote(books: Dict[str, Book], winner_id: str, loser_id: str, k: float = K_FACTOR_DEFAULT):
    a = books[winner_id]
    b = books[loser_id]
    ra_new, rb_new = update_elo(a.rating, b.rating, outcome_a=1, k=k)
    a.rating, b.rating = ra_new, rb_new
    a.wins += 1; a.matches += 1
    b.losses += 1; b.matches += 1


In [10]:
# One-step manual “UI”
a, b = select_pair_nearby(books, window=250)  # or select_pair(...)
print(f"A) {a.title} — {a.author}  [Elo {a.rating:.1f}]")
print(f"B) {b.title} — {b.author}  [Elo {b.rating:.1f}]")

# Decide winner by typing 'a' or 'b' in the next line:
choice = 'a'  # change to 'b' manually per run
if choice == 'a':
    apply_vote(books, a.id, b.id)
else:
    apply_vote(books, b.id, a.id)

save_state(books, STATE_PATH)


A) The Catcher in the Rye — J.D. Salinger  [Elo 1200.0]
B) Brave New World — Aldous Huxley  [Elo 1200.0]


In [11]:
a

Book(id='b5', title='The Catcher in the Rye', author='J.D. Salinger', rating=1216.0, wins=1, losses=0, matches=1)

In [12]:
def simulate(books: Dict[str, Book], n: int = 200):
    ids = list(books.keys())
    for _ in range(n):
        a_id, b_id = random.sample(ids, 2)
        # Simulate true skill ~ current Elo (softened): A wins with prob = expected_score
        pa = expected_score(books[a_id].rating, books[b_id].rating)
        winner = a_id if random.random() < pa else b_id
        loser = b_id if winner == a_id else a_id
        apply_vote(books, winner, loser)
    save_state(books, STATE_PATH)

simulate(books, n=100)


In [13]:
top = sorted(books.values(), key=lambda x: x.rating, reverse=True)
for i, bk in enumerate(top[:10], start=1):
    print(f"{i:>2}. {bk.title:35} {bk.author:22} Elo={bk.rating:7.1f}  (W{bk.wins}-L{bk.losses})")


 1. The Great Gatsby                    F. Scott Fitzgerald    Elo= 1252.5  (W20-L10)
 2. 1984                                George Orwell          Elo= 1230.1  (W12-L12)
 3. The Catcher in the Rye              J.D. Salinger          Elo= 1221.8  (W17-L14)
 4. To Kill a Mockingbird               Harper Lee             Elo= 1212.5  (W12-L10)
 5. Fahrenheit 451                      Ray Bradbury           Elo= 1211.6  (W13-L14)
 6. Dune                                Frank Herbert          Elo= 1209.6  (W10-L11)
 7. The Hobbit                          J.R.R. Tolkien         Elo= 1140.2  (W10-L14)
 8. Brave New World                     Aldous Huxley          Elo= 1121.7  (W7-L16)


In [14]:
# Optional (if pandas installed)
import pandas as pd
df = pd.DataFrame([asdict(b) for b in books.values()]).sort_values("rating", ascending=False)
df.head(10)


,id,title,author,rating,wins,losses,matches
7,b8,The Great Gatsby,F. Scott Fitzgerald,1252.457912,20,10,30
0,b1,1984,George Orwell,1230.083481,12,12,24
4,b5,The Catcher in the Rye,J.D. Salinger,1221.838248,17,14,31
5,b6,To Kill a Mockingbird,Harper Lee,1212.499140,12,10,22
6,b7,Fahrenheit 451,Ray Bradbury,1211.553106,13,14,27
3,b4,Dune,Frank Herbert,1209.621271,10,11,21
2,b3,The Hobbit,J.R.R. Tolkien,1140.246283,10,14,24
1,b2,Brave New World,Aldous Huxley,1121.700558,7,16,23


In [15]:
# Ratings move in the correct direction after a decisive outcome
ra0, rb0 = 1200.0, 1200.0
ra1, rb1 = update_elo(ra0, rb0, outcome_a=1)
assert ra1 > ra0 and rb1 < rb0

# Expected is symmetric
assert abs(expected_score(1400, 1200) + expected_score(1200, 1400) - 1.0) < 1e-9

# State round-trip
save_state(books, STATE_PATH)
books2 = load_state(STATE_PATH)
assert set(books2.keys()) == set(books.keys())
